# 함수 사용하기

## chat.completions는 RAG, 실시간 정보검색기능을 지원 안함

In [23]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "지금 서울의 날씨를 알려줘."}
    ]
)
print(response.choices[0].message.content)

죄송하지만, 현재 실시간 날씨 정보를 제공할 수 없습니다. 서울의 날씨를 확인하려면 날씨 예보 웹사이트나 앱을 이용해 보시는 것을 추천드립니다. 궁금한 점이 있으면 다른 질문도 해주세요!


## 함수 사용하기

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests # pip install requests
import json

# 환경 변수 로드
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
weather_api_key = os.getenv("OPENWEATHER_API_KEY")

In [2]:
client = OpenAI(api_key=openai_api_key)

In [3]:
# 함수 정의
functions = [
    {
        "name": "get_weather",
        "description": "특정 도시의 날씨 정보를 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 가져올 도시의 이름을 영어로"
                }
            },
            "required": ["city"]
        }
    }
]

In [4]:
# 날씨 정보 가져오는 함수
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
    response = requests.get(url) # response는 HTTP 응답
    data = response.json()
    weather = data['weather'][0]['description']
    temperature = data['main']['temp']
    return {
        "city": city,
        "weather": weather,
        "temperature": temperature
    }

In [5]:
# OpenAI API 호출 및 함수 호출 여부 결정
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "서울의 날씨가 궁금해"}
    ],
    functions=functions,
    function_call="auto"
)
print(response)

ChatCompletion(id='chatcmpl-B7zlXNS1754uEUJqqT7eXb2GcsEMs', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"city":"Seoul"}', name='get_weather'), tool_calls=None))], created=1741245423, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=16, prompt_tokens=67, total_tokens=83, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [6]:
# 함수 호출이 발생한 경우
message = response.choices[0].message
if message.function_call and message.function_call.name == 'get_weather':
    arguments = json.loads(response.choices[0].message.function_call.arguments)
    city = arguments['city']
    weather_data = get_weather(city)

    # 날씨 정보를 포함한 최종 응답 출력
    print(f"현재 {city}의 날씨는 {weather_data['weather']}이고, 온도는 {weather_data['temperature']}도 입니다.")
else:
    print(response.choices[0].message.content)

현재 Seoul의 날씨는 clear sky이고, 온도는 8.5도 입니다.


In [21]:
# OpenAI API 호출 및 함수 호출 여부 결정
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "스프링 프레임워크를 만들 사람 이름이 궁금해"}
    ],
    functions=functions,
    function_call="auto"
)
print(response)

ChatCompletion(id='chatcmpl-B7ymoDUlcPbbO6tpKxSjSliQljD1H', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='스프링 프레임워크(SPRING Framework)는 로드 존슨(Rod Johnson)에 의해 개발되었습니다. 그는 2002년에 처음으로 스프링 프레임워크를 공개하였고, 이후 이 프레임워크는 많은 개발자들에 의해 널리 사용되고 있습니다. 현재 스프링 프레임워크는 다수의 커뮤니티 기여자들과 기업들이 함께 관리하고 발전시키고 있습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1741241658, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=95, prompt_tokens=74, total_tokens=169, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [22]:
# 함수 호출이 발생한 경우
if response.choices[0].message.function_call and response.choices[0].message.function_call.name == 'get_weather':
    arguments = json.loads(response.choices[0].message.function_call.arguments)
    city = arguments['city']
    weather_data = get_weather(city)

    # 날씨 정보를 포함한 최종 응답 출력
    print(f"현재 {city}의 날씨는 {weather_data['weather']}이고, 온도는 {weather_data['temperature']}도 입니다.")
else:
    print(response.choices[0].message.content)

스프링 프레임워크(SPRING Framework)는 로드 존슨(Rod Johnson)에 의해 개발되었습니다. 그는 2002년에 처음으로 스프링 프레임워크를 공개하였고, 이후 이 프레임워크는 많은 개발자들에 의해 널리 사용되고 있습니다. 현재 스프링 프레임워크는 다수의 커뮤니티 기여자들과 기업들이 함께 관리하고 발전시키고 있습니다.
